In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-books-africa/book/Pinguilly_Yves Contes_et_légendes_L Afrique_d ouest_en_est.pdf
/kaggle/input/llm-books-africa/book/L enfant Noir Camara Laye.pdf
/kaggle/input/llm-books-africa/book/La victoire des vaincus Jean Ziegler.pdf
/kaggle/input/llm-books-africa/book/Pinguilly Yves Contes_et_légendes_La_Corne_de_l Afrique.pdf
/kaggle/input/llm-books-africa/book/Le Mvett III.pdf
/kaggle/input/llm-books-africa/book/Les Soleils des independances Ahmadou Kourouma.pdf
/kaggle/input/llm-books-africa/book/Le_fabuleux_et_triste_destin_d’Ivan_et_d’Ivana_Condé_Maryse.pdf
/kaggle/input/llm-books-africa/book/Condé Maryse Conte cruel.pdf
/kaggle/input/llm-books-africa/book/La belle Créole - Condé Maryse.pdf
/kaggle/input/llm-books-africa/book/Le pagne noir contes africains.pdf
/kaggle/input/llm-books-africa/book/Vallerey_Gisèle Contes_et_Legendes_de_lAfrique_Noire.pdf
/kaggle/input/llm-books-africa/book/L étrange destin de wangrin  Hamadou Hampaté Bâ.pdf
/kaggle/input/llm-books-africa/boo

In [2]:
pip install PyPDF2 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


# import libraries

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Étape 1: Extraction du texte depuis les PDF
import os
from PyPDF2 import PdfReader

# Étape 2: Nettoyage et prétraitement du texte
import re
from nltk.tokenize import word_tokenize
import torch
import warnings 
warnings.filterwarnings('ignore')

# Data extraction

In [4]:
def extract_text_from_pdfs(pdf_dir):
    texts = []
    for filename in os.listdir(pdf_dir):
        if filename.endswith('.pdf'):
            reader = PdfReader(os.path.join(pdf_dir, filename))
            text = ''
            for page in reader.pages:
                text += page.extract_text()
            texts.append(text)
    return texts

pdf_dir = '/kaggle/input/llm-books-africa/book'
texts = extract_text_from_pdfs(pdf_dir)


# Data Preparation

In [5]:
def preprocess_text(text):
    # Suppression des caractères spéciaux
    text = re.sub(r'\W+', ' ', text)
    # Mise en minuscule
    text = text.lower()
    # Tokenisation
    tokens = word_tokenize(text)
    return ' '.join(tokens)

In [6]:
cleaned_texts = [preprocess_text(text) for text in texts]

# Étape 3: Sauvegarder les données nettoyées pour l'entraînement
train_file = 'train.txt'
with open(train_file, 'w') as f:
    for text in cleaned_texts:
        f.write(text + '\n')

# Load model

In [7]:
# Étape 4: Chargement du modèle pré-entraîné
model_name = "gpt2"  # Tu peux utiliser un modèle plus grand selon ta capacité
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
# Étape 5: Préparation des données pour l'entraînement
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_file,
    block_size=128  # Taille des blocs de texte
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Pour ne pas faire de masked language modeling
)

Token indices sequence length is longer than the specified maximum sequence length for this model (1701818 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
# Étape 6: Fine-tuning du modèle
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,4.088000
1000,3.774800
1500,3.624300
2000,3.523600
2500,3.438100
3000,3.403800
3500,3.377100
4000,3.329100
4500,3.311600


TrainOutput(global_step=4986, training_loss=3.5174955451435896, metrics={'train_runtime': 3959.3994, 'train_samples_per_second': 10.073, 'train_steps_per_second': 1.259, 'total_flos': 2605408174080000.0, 'train_loss': 3.5174955451435896, 'epoch': 3.0})

In [12]:
# Étape 7: Génération de texte
input_text = "raconte l'histoire d'un enfant qui lave un pagne"
#input_ids = tokenizer.encode(input_text, return_tensors='pt')
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

In [13]:
#torch.backends.cuda.enable_mem_efficient_sdp(False)
#torch.backends.cuda.enable_flash_sdp(False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to the correct device
model = model.to(device)

# Ensure input tensors are on the same device as the model
input_ids = input_ids.to(device)

In [14]:
# Modifie les hyperparamètres pour influencer la génération
output = model.generate(
    input_ids,
    max_length=100,
    temperature=0.7,  # Contrôle la créativité du modèle
    top_k=50,  # Limite les choix du modèle à top_k options
    top_p=0.95,  # Utilisation de la "nucleus sampling" pour plus de diversité
    repetition_penalty=1.2  # Réduit la répétition
)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [15]:
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

raconte l'histoire d'un enfant qui lave un pagne de la révolution des épaules et des épaules sont les autres à son côté ils ne se rend jamais vivre pourquoi leur fantait elle avait été déjà avec une grande mère ouverture quelque chose nous allons pas que je suis t
